In [6]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from tabGAN import TabGAN
from src import constants as const
import utils
from utils.tictoc import *
from utils.timer import Timer

imports_path = "src//imports.ipynb"
%run "$imports_path"

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in consts")

In [8]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [9]:
n_epochs = 100
n_critic = 10
opt_lr = 0.002
adam_beta1 = 0.5
noise_discrete_unif_max = 0

batch_size = 500
ckpt_every = 1
loss_plot_update_every = 10

timer_path = os.path.join(const.dir.timers(), "timer_tabGAN_speed.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = utils.timer.Timer()
    timer.save(timer_path)
    del timer

In [11]:
tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False)

In [22]:
tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(tg_qtr.data_discrete_oh),
                     tf.data.Dataset.from_tensor_slices(tg_qtr.data_num))
                   )

<ZipDataset shapes: ((44,), (6,)), types: (tf.float64, tf.int64)>

In [32]:
from tabGAN import TabGAN2
tg_qtr2 = TabGAN2(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False)

In [35]:
tg_qtr2.data_processed

<ZipDataset shapes: ((6,), (44,)), types: (tf.int64, tf.float64)>

In [44]:
for b in tg_qtr2.data_processed.shuffle(buffer_size=tg_qtr2.data.shape[0]).batch(500).prefetch(2):
    print(b)

(<tf.Tensor: shape=(500, 6), dtype=int64, numpy=
array([[    40,  95226,     13,      0,      0,     50],
       [    24, 346909,      9,      0,      0,     44],
       [    43, 225165,      7,      0,      0,     55],
       ...,
       [    56, 211804,     13,  15024,      0,     50],
       [    22,  50065,     13,      0,      0,     35],
       [    18,  45316,      7,      0,      0,     20]], dtype=int64)>, <tf.Tensor: shape=(500, 44), dtype=float64, numpy=
array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 1.],
       [1., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 1., 1., 0.]])>)
(<tf.Tensor: shape=(500, 6), dtype=int64, numpy=
array([[    46, 187581,      9,      0,      0,     40],
       [    37, 108366,     13,      0,      0,     46],
       [    33, 857532,      8,      0,      0,     40],
       ...,
       [    40, 262872,     13,      0,      0,     40]

In [16]:
dat = pd.read_csv("V:/hyperparams_tuning/tabGAN-qtr/oh_encoding_choices_comparison/oh_encoding_choices_noise_discrete_unif_max_0.0_gumbel_temp_1/gen6.csv")

In [17]:
dat.isna().sum()

Unnamed: 0         0
age                0
workclass          0
fnlwgt             0
education          0
educational.num    0
marital.status     0
occupation         0
race               0
gender             0
capital.gain       0
capital.loss       0
hours.per.week     0
native.country     0
income             0
dtype: int64